# DARPA-ARC Notebook 6: fMRI Figures

## Figure 3: Mindboggle Parcellation

In [ ]:
import numpy as np
import os.path as op
from mne import read_label
from surfer import Brain
from seaborn import color_palette
#%matplotlib qt5

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
fs_dir = '/space/lilli/1/users/DARPA-Recons'
version = 'Version20190405'
results_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
subject = 'fsaverage'
surface = 'inflated'
hemi = 'lh'

## Label paramaters.
label_dir = '/space/lilli/1/users/DARPA-Recons/fscopy/label/dkt40'
roi_list = ['caudalanteriorcingulate', 'rostralanteriorcingulate', 'posteriorcingulate',
            'superiorfrontal', 'medialorbitofrontal', 'rostralmiddlefrontal', 'caudalmiddlefrontal',
            'parsopercularis', 'parstriangularis', 'parsorbitalis', 'lateralorbitofrontal', 'insula']

colors = [[0.78329874347238, 0.687243385525311, 0.8336793640080622],
          [0.1257208769520124, 0.47323337360924367, 0.707327968232772],
            [0.999907727802501, 0.5009919264737298, 0.005121107311809869],
          [0.21171857311445125, 0.6332641510402455, 0.1812226118410335],
          [0.6941176652908325, 0.3490196168422699, 0.1568627506494522],
          [0.42485198495434734, 0.2511495584950722, 0.6038600774372326],
          [0.983206460055183, 0.5980161709820524, 0.5942330108845937],
          [0.9917570170234231, 0.7464821371669862, 0.4340176893507733],
          [0.8905959311653586, 0.10449827132271793, 0.111080354627441],
          [0.9976009228650261, 0.9948942715046452, 0.5965244373854468],
          [0.6889965575115352, 0.8681737867056154, 0.5437601194662207],
          [0.6509804129600525, 0.8078431487083435, 0.8901960849761963]]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Visualize brain.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load labels.
roi_list = [read_label('%s/%s.%s.label' %(label_dir, hemi, roi), subject=subject) for roi in roi_list]

## Make brain.
brain = Brain(subject, hemi, surface, background='white', subjects_dir=fs_dir)
for roi, color in zip(roi_list, colors): brain.add_label(roi, color=color)
for view in ['medial','lateral']:
    brain.show_view(view)
    brain.save_image(op.join(results_dir, 'plots/%s/mindboggle_%s.png' % (version, view)))

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

version = 'Version20190405'
results_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'

for view in ['medial','lateral']:
    imgf = op.join(results_dir, 'plots/%s/mindboggle_%s.png' % (version, view))
    img = Image.open(imgf)
    img = trim(img)
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.savefig(op.join(results_dir, 'plots/%s/mindboggle_%s_high_res.png' % (version, view)), dpi = 300, bbox_info='tight')
    plt.close('all')

## Figure 6: 6-panel Deliberation Map

In [ ]:
import os, time
import os.path as op
import numpy as np
from surfer import Brain

fs_dir = '/autofs/space/lilli_001/users/DARPA-Recons/'
root_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
version = 'Version20190405'
img_dir = op.join(root_dir, 'plots/%s/manuscript' % version)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#


## I/O parameters.
sm = 6
fd = 0.9
contrasts = ['Delib', 'DelibMod', 'FixedEpochs', 'Risk', 'Reward']
overlay = 'psc'

## Pysurfer parameters.
surface = 'inflated'
views = ['lateral','medial']
size = (1200,800)
bg_color = 'black'

fmin = 0.04 # min value
fmax = 0.20 # max value

## Label parameters.
for contrast in contrasts:
    label_dir = op.join(root_dir, 'fmri_second_levels/labels/seeds_%s' % contrast)
    labels = ['dacc', 'dlpfc', 'insula', 'mcc', 'pre_sma']
    label_color = 'k'
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    ## Initialize brain.
    brains = Brain("fsaverage", 'split', surface, views=views, size=size,
                   background=bg_color, subjects_dir=fs_dir)
    #
    ## Add overlay.
    for hemi in ['lh','rh']:
        f = op.join(root_dir, 'fmri_second_levels', '%s.%s.%s.%s.%s' %(version, sm, fd, hemi, contrast), '%s.nii.gz' % overlay)
        brains.add_overlay(f, min=fmin, max=fmax, sign="pos", hemi=hemi)
    #
    ## Add labels.
    # for label in labels:
    #     for hemi in ['lh','rh']:
    #         f = os.path.join(label_dir, 'fig_%s-%s.label' %(label,hemi))
    #         brains.add_label(f, color=label_color, borders=True, hemi=hemi)
    #
    ## Update views.
    brains.set_distance(300)
    brains.hide_colorbar(0,0)
    brains.hide_colorbar()
    #
    time.sleep(5)
    if not os.path.isdir(img_dir): os.makedirs(img_dir)
    brains.save_image(op.join(img_dir, '%s_surface.png' % contrast))

In [5]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os.path as op
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

version = 'Version20190405'
root_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
img_dir = op.join(root_dir, 'plots/%s/manuscript' % version)

contrasts = ['Delib', 'DelibMod', 'FixedEpochs', 'Risk', 'Reward']

for contrast in contrasts:
    imgf = op.join(img_dir, '%s_surface.png' % contrast)
    img = Image.open(imgf)
    img = trim(img)
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.savefig(op.join(img_dir, '%s_surface_high_res.png' % contrast), dpi = 300, bbox_info='tight')
    plt.close('all')

## Figure 7: 3-panel Delibmod

In [ ]:
import os, time
import os.path as op
import numpy as np
from surfer import Brain

fs_dir = '/autofs/space/lilli_001/users/DARPA-Recons/'
root_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
version = 'Version20190405'
img_dir = op.join(root_dir, 'plots/%s/manuscript' % version)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
sm = 6
fd = 0.9
contrasts = ['Delib', 'DelibMod', 'FixedEpochs', 'Risk', 'Reward']
overlay = 'psc'

## Pysurfer parameters.
surface = 'pial'
size = (800,400)
bg_color = 'black'

fmin = 1e-6 # min value
fmax = 0.10 # max value

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

hemis = ['lh','rh','rh']
views = ['lateral','lateral','medial']
params = [dict(azimuth=150, roll=90),dict(azimuth=30, roll=270),dict(azimuth=180, roll=90)]
distances = [280,280,280]

for contrast in contrasts:
    #
    for hemi, view, param, dist in zip(hemis,views,params,distances):
        #
        ## Initialize brain.
        brain = Brain("fsaverage", hemi, surface, views=[view], size=size,
                      background=bg_color, subjects_dir=fs_dir)
        #
        ## Add overlay.
        f = op.join(root_dir, 'fmri_second_levels', '%s.%s.%s.%s.%s' % (version, sm, fd, hemi, contrast), '%s.nii.gz' % overlay)
        brain.add_overlay(f, min=fmin, max=fmax, sign="pos", hemi=hemi)
        #
        ## Update views.
        brain.show_view(param)
        brain.set_distance(dist)
        brain.hide_colorbar()
        time.sleep(5)
        brain.save_image(op.join(img_dir, '%s_%s_%s.png' % (contrast, hemi, view)))
    
print('Done.')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os.path as op
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

version = 'Version20190405'
root_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
img_dir = op.join(root_dir, 'plots/%s/manuscript' % version)

contrasts = ['Delib', 'DelibMod', 'FixedEpochs', 'Risk', 'Reward']
hemis = ['lh','rh','rh']
views = ['lateral','lateral','medial']

for contrast in contrasts:
    for hemi in hemis:
        for view in views:
            imgf = op.join(img_dir, '%s_%s_%s.png' % (contrast, hemi, view))
            img = Image.open(imgf)
            img = trim(img)
            imgplot = plt.imshow(img)
            plt.axis('off')
            plt.savefig(op.join(img_dir, '%s_%s_%s_high_res.png' % (contrast, hemi, view)), dpi = 300, bbox_info='tight')
            plt.close('all')

### Pysurfer Colorbar

In [ ]:
import os 
import os.path as op
import numpy as np
from surfer import Brain

fs_dir = '/autofs/space/lilli_001/users/DARPA-Recons/'
root_dir = '/autofs/space/lilli_002/users/JNeurosci_ARC/'
version = 'Version20190405'
img_dir = op.join(root_dir, 'plots/%s/manuscript' % version)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Pysurfer parameters.
size = (1200,800)
bg_color = 'black'

fmin = 0.00 # min value
fmax = 0.20 # max value

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Initialize brain.
brains = Brain("fsaverage", 'lh', 'inflated', size=size,
              background=bg_color, subjects_dir=fs_dir)

## Add overlay.    
f = op.join(root_dir, 'fmri_second_levels/%s.6.0.9.lh.Delib/psc.nii.gz' % version)
brains.add_overlay(f, min=fmin, max=fmax, sign="pos")